### pymupdf4llm

In [ ]:
import pymupdf4llm
import pathlib

md_text = pymupdf4llm.to_markdown('C:/Users/EL0017/Desktop/ㅋ/새 폴더/2025년1차청년매입임대입주자모집공고문(광주전남).pdf')  # PDF → Markdown 텍스트
pathlib.Path("test.md").write_bytes(md_text.encode())  # 저장


### pdf -> html -> md (html 경유)

In [ ]:
import fitz  # PyMuPDF
import html2text

def pdf_to_markdown_via_html(pdf_path, md_path):
    doc = fitz.open(pdf_path)
    html_content = ""

    for page in doc:
        html_content += page.get_text("html")

    markdown = html2text.html2text(html_content)

    with open(md_path, "w", encoding="utf-8") as f:
        f.write(markdown)


In [ ]:
pdf = 'E:/work/MS_project_2/data/test.pdf'
md = 'E:/work/MS_project_2/data/test2.md'
pdf_to_markdown_via_html(pdf,md)

### pdfplumber

In [ ]:
import pdfplumber
import json

pdf_path = "E:/work/MS_project_2/data/test.pdf"  # 너의 PDF 경로로 바꿔줘
output_path = "E:/work/MS_project_2/data/test.json"

results = []

with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        page_data = {
            "page": page_num,
            "text": page.extract_text(),  # 일반 텍스트
            "tables": []
        }

        # 표 추출 (리스트 of 리스트 구조)
        tables = page.extract_tables()
        for table in tables:
            page_data["tables"].append(table)

        results.append(page_data)

# ✅ 결과를 JSON 파일로 저장 (표와 텍스트를 같이 저장)
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"✅ 변환 완료! {output_path}에 저장됨")


### DI -> json -> markdown

In [ ]:
import requests
import time
import json

EDNPOINT = ''
API_KEY = ''


### json파일 생성
def request_document_intelligence(file_path):
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/pdf'
    }

    try:
        with open(file_path, 'rb') as f:
            file_data = f.read()
    except FileNotFoundError:
        print(f"[오류] 파일을 찾을 수 없습니다: {file_path}")
        return None

    print("[요청 전송 중] Document Intelligence 분석 요청을 보냅니다...")
    response = requests.post(EDNPOINT, headers=headers, data=file_data)

    if response.status_code != 202:
        print(f"[에러] 요청 실패! 상태 코드: {response.status_code}")
        print(response.text)
        return None

    result_url = response.headers['Operation-Location']
    print(f"[요청 수락됨] 결과를 기다립니다... Operation URL: {result_url}")

    # 결과가 준비될 때까지 폴링
    while True:
        result_response = requests.get(result_url, headers={
            'Ocp-Apim-Subscription-Key': API_KEY
        })

        result_json = result_response.json()
        status = result_json.get("status", "")

        if status.lower() in ["running", "notStarted"]:
            print(f"[대기 중] 현재 상태: {status}")
            time.sleep(1)
        elif status.lower() == "succeeded":
            print("[완료] 분석이 성공적으로 완료되었습니다.")
            return result_json
        else:
            print(f"[오류] 분석 실패 또는 예외 발생: {status}")
            print(result_json)
            return None


# 🚀 실행 예시
pdf_path = 'test.pdf'  # 분석할 PDF 파일 경로
result = request_document_intelligence(pdf_path)

# 결과 저장 (선택 사항)
if result:
    import json
    with open("document_intelligence_result.json", "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    print("✅ 결과가 'document_intelligence_result.json'에 저장되었습니다.")

In [1]:
import requests
import time
import json

EDNPOINT = ''
API_KEY = ''


def request_document_intelligence(file_path):
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/pdf'
    }

    with open(file_path, 'rb') as f:
        file_data = f.read()

    print("[요청 전송 중] Document Intelligence 분석 요청을 보냅니다...")
    response = requests.post(EDNPOINT, headers=headers, data=file_data)

    if response.status_code != 202:
        print(f"[에러] 요청 실패! 상태 코드: {response.status_code}")
        print(response.text)
        return None

    result_url = response.headers['Operation-Location']
    print(f"[요청 수락됨] 결과를 기다립니다... Operation URL: {result_url}")

    # 결과가 준비될 때까지 폴링
    while True:
        result_response = requests.get(result_url, headers={
            'Ocp-Apim-Subscription-Key': API_KEY
        })

        result_json = result_response.json()
        status = result_json.get("status", "")

        if status.lower() in ["running", "notStarted"]:
            print(f"[대기 중] 현재 상태: {status}")
            time.sleep(1)
        elif status.lower() == "succeeded":
            print("[완료] 분석이 성공적으로 완료되었습니다.")
            return result_json
        else:
            print(f"[오류] 분석 실패 또는 예외 발생: {status}")
            print(result_json)
            return None


def convert_to_markdown(result_json):
    """JSON 결과를 Markdown 형식으로 변환"""
    md_lines = []
    pages = result_json.get("analyzeResult", {}).get("pages", [])

    for page in pages:
        md_lines.append(f"# 📄 Page {page['pageNumber']}\n")

        # ✅ 1. 일반 텍스트 라인
        lines = page.get("lines", [])
        for line in lines:
            text = line.get("content", "")
            md_lines.append(text)
        md_lines.append("\n---\n")

        # ✅ 2. 테이블 처리
        tables = page.get("tables", [])
        for t_idx, table in enumerate(tables):
            md_lines.append(f"\n## 📊 Table {t_idx + 1}\n")

            row_dict = {}
            for cell in table.get("cells", []):
                row_index = cell["rowIndex"]
                col_index = cell["columnIndex"]
                content = cell["content"]

                if row_index not in row_dict:
                    row_dict[row_index] = {}
                row_dict[row_index][col_index] = content

            max_col = max(max(row.keys()) for row in row_dict.values())

            # 표 작성
            for row_index in sorted(row_dict.keys()):
                row = row_dict[row_index]
                line = " | ".join([row.get(i, "") for i in range(max_col + 1)])
                md_lines.append(f"| {line} |")

                if row_index == 0:
                    md_lines.append("|" + " --- |" * (max_col + 1))

        md_lines.append("\n---\n")

    return "\n".join(md_lines)


# 🚀 실행 예시
pdf_path = 'E:/work/MS_project_2/data/test.pdf'  # 분석할 PDF 파일 경로
result = request_document_intelligence(pdf_path)

if result:
    markdown = convert_to_markdown(result)

    with open("E:/work/MS_project_2/data/test_di.md", "w", encoding="utf-8") as f:
        f.write(markdown)

    print("✅ Markdown으로 저장 완료: document_intelligence_result.md")


FileNotFoundError: [Errno 2] No such file or directory: 'E:/work/MS_project_2/data/test.pdf'


1
공급개요
■ 공급대상 주택 : 총 190호
• 주택군, 주택별 소재지, 면적, 임대조건 등 세부내역은 첨부 “주택내역” 참조
* 위 공급호수는 기존에 선정된 예비입주자 계약 및 신규 매입물량 추가 등으로 변경될 수 있습니다.
* 청약플러스(https://apply.lh.or.kr) 공고문 페이지에 주택사진이 게시되어 있으므로 참고하시기 바랍니다.
* 아동복지시설(가정위탁 포함) 및 청소년쉼터 퇴소자 수시 우선공급으로 인해 지역별 공급물량이
감소될 수 있습니다.
* 본 모집공고의 예비입주자로 선정되었다 하더라도 기존 임차인의 퇴거 및 임대공급 시행 전 실시하는
주택 개보수 완료 상황 등에 따라 주택물량이 감소하거나 입주까지 상당기간이 소요될 수 있습니다.
■ 임대기간 및 임대조건
구분
내용
임대기간
• 최초 임대기간은 2년이며, 자격 충족 시 재계약(2년 단위) 4회 가능(최장 10년 거주)
- 입주 후 혼인한 경우, 재계약(2년 단위) 5회 추가연장 가능(최장 20년 거주)
임대조건
• 시중 시세 40~50% 수준의 임대료 및 임대보증금으로 입주 순위에 따라 차등
• 공급대상 주택별 임대조건은 첨부 "주택내역" 참조
구분
1순위
2 · 3순위
임대조건
시중 시세의 40%
(임대보증금 100만원)
시중 시세의 50%
(임대보증금 200만원)
임대보증금-
월임대료
전환
• 임대보증금을 10만원 단위로 증액하여 월임대료를 낮출 수 있음
- (전환한도) 기본 월임대료의 최대 60%를 보증금으로 전환 가능
(단, 월임대료는 주택별 월임대료 하한기준액 이하로 낮아질 수 없음)
- (증액 전환이율) 연 7%
- (계산방법) 임대보증금 증액분 × 7% ÷ 12개월 = 월임대료 감소분
※ (예시) 임대보증금을 300만원 추가 납부할 경우 월임대료 17,500원 감소
* 재계약 시 관계법령이 정한 범위 내에서 임대보증금 및 월임대료가 인상될 수 있으며 전환이율, 하한
기준액 등은 실제 적용 시점에 변동될 수 있습니다.
* 관리업무(공용부분 청소, 공과금 배분 등)는 외부 위탁기관(관리사무소)에서 담당하고 있으며, 별도
청소용역비 및 관리비를 부담하셔야 합니다.
* 오피스텔· 도시형생활주택 · 아파트 등은 다가구·다세대주택에 비해 관리비(세대 개별 사용분 제외)가 두배
이상 높을 수 있으니 신청 시 양지하시기 바랍니다.
* 청년 매입임대주택에 입주한 적이 있는 사람이 다른 청년 매입임대주택을 신청하여 입주하는 경우에는
종전 청년 매입임대주택의 계약횟수 및 임대차 기간이 차감됩니다.

### DI -> Markdown

In [ ]:
import os
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
from azure.core.credentials import AzureKeyCredential
from datetime import datetime, timedelta

# ✅ 1. 설정
BLOB_CONN_STR = ""
BLOB_CONTAINER_NAME = "pdf-container"
PDF_LOCAL_PATH = "C:/Users/EL0017/Desktop/ㅋ/새 폴더/2025년1차청년매입임대입주자모집공고문(광주전남).pdf"
#PDF_LOCAL_PATH = "E:/work/MS_project_2/data/test.pdf"
PDF_BLOB_NAME = "uploaded_test.pdf"

DI_ENDPOINT = ""
DI_API_KEY = ''

# ✅ 2. Blob Storage에 업로드
blob_service = BlobServiceClient.from_connection_string(BLOB_CONN_STR)
container_client = blob_service.get_container_client(BLOB_CONTAINER_NAME)

# 컨테이너가 없다면 생성
if not container_client.exists():
    container_client.create_container()

blob_client = container_client.get_blob_client(PDF_BLOB_NAME)

# PDF 파일 업로드
with open(PDF_LOCAL_PATH, "rb") as f:
    blob_client.upload_blob(f, overwrite=True)
print("✅ PDF 업로드 완료")

# ✅ 3. SAS URL 생성 (15분 유효)
sas_token = generate_blob_sas(
    account_name=blob_service.account_name,
    container_name=BLOB_CONTAINER_NAME,
    blob_name=PDF_BLOB_NAME,
    account_key=blob_service.credential.account_key,
    permission=BlobSasPermissions(read=True),
    expiry=datetime.utcnow() + timedelta(minutes=15)
)

sas_url = f"{blob_client.url}?{sas_token}"
print("✅ SAS URL 생성 완료")

# ✅ 4. Document Intelligence Markdown 분석
client = DocumentIntelligenceClient(endpoint=DI_ENDPOINT, credential=AzureKeyCredential(DI_API_KEY))

poller = client.begin_analyze_document(
    model_id="prebuilt-layout",
    body=AnalyzeDocumentRequest(url_source=sas_url),
    output_content_format='markdown'
)

result = poller.result()

# ✅ 5. 결과 저장
with open("document_result.md", "w", encoding="utf-8") as f:
    f.write(result.content)
print("✅ Markdown 분석 결과 저장 완료")


서울지역본부 청년 매입임대주택 예비입주자 모집공고
[강남구, 강동구, 강북구, 관악구, 구로구, 금천구, 노원구, 동대문구, 마포구, 서대문구, 서초구,
송파구, 용산구, 은평구, 종로구, 중구]

☐
청년 매입임대주택은 LH에서 주택을 매입하여 청년(만19세~39세), 대학생 및 취업준비생을 대상으로
시중시세 40~50% 수준으로 임대하는 주택입니다.

☐
LH에서는 마이홈센터(☎1600-1004, 내선번호 2번→3번) 및 서울지역본부 매입임대 상담센터(☏
02-2015-1040)를 통해 모집공고에 대한 안내가 이루어질 수 있도록 상담을 실시하고 있습니다. (다만,
상담내용은 신청 참고자료로만 활용하여 주시고, 반드시 공고문을 통해 세부 내용을 숙지하신 후
신청하여 주시기 바랍니다.). 아울러, 신청자격 미숙지, 착오신청 등에 대해서 신청자 본인에게 책임이 있
으니 유의하여 주시기 바랍니다.
□

■ (기준일) 모집공고일은 2025.03.27.(목)이며, 이는 입주자격(신청자격, 나이, 세대구성원, 주택소유,
자산, 소득 등)의 판단기준일이 됩니다.


# ■ (모집호수) 총 190호, ■ (주택관리번호) 2025-980038

■ (모집일정)

\* 일정은 진행상황에 따라 변동될 수 있습니다.


<table>
<tr>
<th>모집 공고</th>
<th rowspan="2">⇨</th>
<th>신청 (PC,모바일)</th>
<th rowspan="2">⇨</th>
<th>서류심사 대상자발표</th>
<th rowspan="2">⇨</th>
<th>서류제출</th>
<th rowspan="2">⇨</th>
<th>자격검증</th>
<th rowspan="2">⇨</th>
<th>예비입주자 순번 발표</th>
<th rowspan="2">⇨</th>
<th>계약체결 (순번 도래시)</th>
<th></th>
<th>입주</th>
</tr>
<tr>
<td>3.27(목)</td>
<td>4.7(월) ~ 4.9(수)</td>
<td>4.11(금)</td>
<td>4.14(월) ~ 4.16(수)</td>
<td>4.17(목) ~ 6.12(목)</td>
<td>6.13(금)</td>
<td>별도 안내</td>
<td>⇨</td>
<td>계약일부터 60일 이내</td>
</tr>
</table>


■ (중복신청 불가) 1인 1주택 신청하여야 하며, 중복 신청하는 경우 전부 무효처리됩니다.
\- 청년 매입임대주택 기계약자는 동일한 지자체(시·군·구) 모집에 중복신청 불가합니다.

■ (가전제품 등 비치) 청년 매입임대주택은 냉장고, 전자레인지, 책걸상 등이 비치되어 있거나 비치될
예정입니다. (비치된 가전 및 가구의 처분은 불가하며 분실·멸실·파손 시 LH에 통보하고 원상복구
또는 원상복구비용을 부담하여야 하며, 주택여건에 따라 비치되는 품목이 변경될 수 있습니다.)

■ (모집단위) 신청은 시·군·구 내 주택군을 모집단위로 하고, 여러 개의 모집단위가 있는 경우 그중
한 개의 모집단위를 선택하여 신청합니다.

■ (성별분리) '공급주택목록'의 '성별용도 구분'에 유의하여 신청하시기 바랍니다.

\* 남성전용 주택에 여성 신청 시 부적격, 여성전용 주택에 남성 신청 시 부적격

■ (모집인원) 예비입주자 모집인원은 공급대상 주택의 3배수입니다. (지역, 주택군에 따라 다를 수 있습니다.)

■ (공급방법) 모집단위별 예비입주자 순번(주택을 지정할 수 있는 순번) 발표 후 해당 모집단위 내 공급 가능한
주택을 개방하고 순번에 따라 희망하는 주택을 지정하여 계약 체결하는 방식으로 공급합니다.

■ (예비입주자 자격) 예비입주자 자격은 모집단위별 예비입주자 순번 발표일로부터 60일간 유지되며,
60일이 경과한 다음 날 예비입주자 지위가 소멸됩니다.

■ (입주지정기간) 계약체결 후 60일 이내에 입주를 완료하여야 합니다.

■ (신청유의) 신청 내용이 사실과 다를 경우 부적격 처리, 당첨 취소 등 불이익이 있을 수 있으며,
이로 인한 책임은 신청자 본인에게 있음을 유의하시어 공고문을 반드시 숙지하신 후 정확하게
신청하여 주시기 바랍니다.


# 공급개요


## ■ 공급대상 주택 : 총 190호

• 주택군, 주택별 소재지, 면적, 임대조건 등 세부내역은 첨부 “주택내역” 참조

\* 위 공급호수는 기존에 선정된 예비입주자 계약 및 신규 매입물량 추가 등으로 변경될 수 있습니다.

\* 청약플러스(https://apply.lh.or.kr) 공고문 페이지에 주택사진이 게시되어 있으므로 참고하시기 바랍니다.

\* 아동복지시설(가정위탁 포함) 및 청소년쉼터 퇴소자 수시 우선공급으로 인해 지역별 공급물량이
감소될 수 있습니다.

\* 본 모집공고의 예비입주자로 선정되었다 하더라도 기존 임차인의 퇴거 및 임대공급 시행 전 실시하는
주택 개보수 완료 상황 등에 따라 주택물량이 감소하거나 입주까지 상당기간이 소요될 수 있습니다.


## ■ 임대기간 및 임대조건


<table>
<tr>
<th>구분</th>
<th colspan="3">내용</th>
</tr>
<tr>
<td>임대기간</td>
<td colspan="3">• 최초 임대기간은 2년이며, 자격 충족 시 재계약(2년 단위) 4회 가능(최장 10년 거주) - 입주 후 혼인한 경우, 재계약(2년 단위) 5회 추가연장 가능(최장 20년 거주)</td>
</tr>
<tr>
<td rowspan="4">임대조건</td>
<td colspan="3">• 시중 시세 40~50% 수준의 임대료 및 임대보증금으로 입주 순위에 따라 차등 • 공급대상 주택별 임대조건은 첨부 "주택내역" 참조</td>
</tr>
<tr>
<td>구분</td>
<td>1순위</td>
<td>2 · 3순위</td>
</tr>
<tr>
<td>임대조건</td>
<td>시중 시세의 40% (임대보증금 100만원)</td>
<td>시중 시세의 50% (임대보증금 200만원)</td>
</tr>
<tr>
<td colspan="3"></td>
</tr>
<tr>
<td>임대보증금- 월임대료 전환</td>
<td colspan="3">• 임대보증금을 10만원 단위로 증액하여 월임대료를 낮출 수 있음 - (전환한도) 기본 월임대료의 최대 60%를 보증금으로 전환 가능 (단, 월임대료는 주택별 월임대료 하한기준액 이하로 낮아질 수 없음) - (증액 전환이율) 연 7% - (계산방법) 임대보증금 증액분 × 7% ÷ 12개월 = 월임대료 감소분 ※ (예시) 임대보증금을 300만원 추가 납부할 경우 월임대료 17,500원 감소</td>
</tr>
</table>